<h1><center>CS 455/595a: Support Vector Machines Demos</center></h1>
Richard S. Stansbury

Reference:

[1] Aurelen Geron. *Hands on Machine Learning with Scikit-Learn & TensorFlow* O'Reilley Media Inc, 2017.

[2] Aurelen Geron. "ageron/handson-ml: A series of Jupyter notebooks that walk you through the fundamentals of Machine Learning and Deep Learning in python using Scikit-Learn and TensorFlow." Github.com, online at: https://github.com/ageron/handson-ml [last accessed 2019-03-01]

**Table of Contents**
* [Linear Support Vector Machines](#Linear-Support-Vector-Machines)
    * [Linear SVM Classifier](#Linear-SVM-Classifier)
    * [Soft Margin Classifier](#Soft-Margin-Classifier) 
 [Non-Linear Support Vector Machines](#Non-Linear-Support-Vector-Machines)
    * [Polynomial Kernel](#Polynomial-Kernel)
    * [Similarity Features](#Similarity-Features)
    * [Gaussian RBF Kernel](#Gaussian-RBF-Kernel)

# Linear Support Vector Machines

## Linear SVM Classifier

## Soft Margin Classifier

# Non-Linear Support Vector Machines

## Polynomia Kernel

## Similarity Features

## Gaussian RBF Kernel